In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))

from trading_bot.engine import TradingEngine
import pandas as pd

bot = TradingEngine()

# Determine testing ranges
testing_ranges = [5, 10, 20, 50, 100, 200]
opt_results = []

# Download data once
df_base = bot.fetch_data(timeframe='1h', limit=1000)

for v in testing_ranges:
    # Do not manipulate original DataFrame
    df_temp = df_base.copy()
    
    # Run backtest with current window
    df_temp = bot.add_indicators(df_temp, sma_window=v)
    df_temp = bot.get_signals(df_temp, sma_window=v)
    res = bot.run_backtest(df_temp)
    
    # Store result
    opt_results.append({
        'Window': v,
        'Return %': res['total_return_pct'],
        'Final $': res['final_balance']
    })

# Convert to DataFrame for comparison
df_results = pd.DataFrame(opt_results)
print(df_results.sort_values(by='Return %', ascending=False))

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))

from trading_bot.engine import TradingEngine
import pandas as pd

symbol = 'BTC/USDT'  # Bitcoin
# symbol = 'LTC/USDT'  # Litecoin
# symbol = 'ETH/USDT'   # Ethereum
# symbol = 'ADA/USDT' # Cardano
# symbol = 'SOL/USDT' # Solana


bot = TradingEngine(symbol=symbol)  

# Download data once
df_base = bot.fetch_data(timeframe='1h', limit=1000)
df_base = bot.add_indicators(df_base, sma_window=10, ema_window=10, rsi_window=14, bb_window=20)

# Define parameter combinations to test
combinations = [
    {'sma_window':10, 'ema_window':None, 'use_macd':False, 'use_bbands':False},
    {'sma_window':10, 'ema_window':10, 'use_macd':False, 'use_bbands':False},
    {'sma_window':10, 'ema_window':10, 'use_macd':True, 'use_bbands':False},
    {'sma_window':10, 'ema_window':10, 'use_macd':True, 'use_bbands':True},
]

results = []

# Iterate over combinations
for params in combinations:
    df_temp = df_base.copy()
    df_temp = bot.get_signals(
        df_temp,
        sma_window=params['sma_window'],
        ema_window=params['ema_window'],
        use_macd=params['use_macd'],
        use_bbands=params['use_bbands'],
        rsi_overbought=60,
        rsi_oversold=50
    )
    # Run backtest
    res = bot.run_backtest(df_temp)
    results.append({
        **params,
        'Return %': res['total_return_pct'],
        'Final $': res['final_balance'],
        'Max Drawdown %': res['df']['drawdown'].min()*100
    })

# Turn into DataFrame for comparison
df_results = pd.DataFrame(results)
df_results.sort_values(by='Return %', ascending=False)


In [ ]:
import mplfinance as mpf
import numpy as np
import pandas as pd
from trading_bot.engine import TradingEngine

symbol = 'BTC/USDT'  # Bitcoin
# symbol = 'LTC/USDT'  # Litecoin
# symbol = 'ETH/USDT'   # Ethereum
# symbol = 'ADA/USDT' # Cardano
# symbol = 'SOL/USDT' # Solana



# Fetch original data
bot = TradingEngine(symbol=symbol)
df = bot.fetch_data(timeframe='1h', limit=1000)
# Compute indicators & signals
df = bot.add_indicators(df, sma_window=20, ema_window=20, rsi_window=14, bb_window=20)
df = bot.get_signals(df, sma_window=20, rsi_overbought=60, rsi_oversold=50, use_macd=True, use_bbands=True)

# Copy and prepare data
df_plot = df.copy()
df_plot.set_index('timestamp', inplace=True)

# Buy/sell signs
df_plot['Signal_Diff'] = df_plot['Signal'].diff()

df_plot['buy_price'] = np.where(df_plot['Signal_Diff'] > 0, df_plot['close'] * 0.98, np.nan)
df_plot['sell_price'] = np.where(df_plot['Signal_Diff'] < 0, df_plot['close'] * 1.02, np.nan)

# Focus on recent data
df_plot_recent = df_plot.tail(300).copy()
df_plot_recent.dropna(subset=['MACD', 'MACD_Signal', 'RSI'], inplace=True)

# Addplots for each indicator

apds = [
    mpf.make_addplot(df_plot_recent['SMA_20'], color='orange', width=1.5, panel=0),
    mpf.make_addplot(df_plot_recent['EMA_20'], color='blue', width=1.5, panel=0),
    mpf.make_addplot(df_plot_recent['BB_High'], color='red', width=0.8, panel=0, alpha=0.5),
    mpf.make_addplot(df_plot_recent['BB_Low'], color='green', width=0.8, panel=0, alpha=0.5),
    mpf.make_addplot(df_plot_recent['MACD'], color='purple', panel=1, ylabel='MACD'),
    mpf.make_addplot(df_plot_recent['MACD_Signal'], color='black', panel=1),
    mpf.make_addplot(df_plot_recent['RSI'], color='brown', panel=2, ylabel='RSI'),
]

if not df_plot_recent['buy_price'].dropna().empty:
    apds.append(mpf.make_addplot(df_plot_recent['buy_price'], type='scatter', markersize=100, marker='^', color='green', panel=0))

if not df_plot_recent['sell_price'].dropna().empty:
    apds.append(mpf.make_addplot(df_plot_recent['sell_price'], type='scatter', markersize=100, marker='v', color='red', panel=0))


# Plot
mpf.plot(df_plot_recent,
         type='candle',
         style='charles',
         title=f"\n{bot.symbol} - Analysis",
         ylabel='Price (USDT)',
         volume=True,
         addplot=apds,
         figsize=(16,10),
         panel_ratios=(3,1,1),
         tight_layout=True)

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))


import time
import numpy as np
import pandas as pd
import ta  # TA version
from trading_bot import trading_core


# 1. Generate data (1 million candles)
print("Generating data...")
data = 100 + np.random.randn(1_000_000).cumsum()
df_bench = pd.DataFrame(data, columns=['close'])

def add_indicators_python(df, sma_window=20, ema_window=20, rsi_window=14, 
                          macd_fast=12, macd_slow=26, macd_signal=9, bb_window=20):
    """Pandas and TA library version"""
    d = df.copy()
    # SMA/EMA
    d[f'SMA_{sma_window}'] = d['close'].rolling(window=sma_window).mean()
    d[f'EMA_{ema_window}'] = d['close'].ewm(span=ema_window, adjust=False).mean()
    # RSI
    d['RSI'] = ta.momentum.RSIIndicator(d['close'], window=rsi_window).rsi()
    # MACD
    macd = ta.trend.MACD(d['close'], macd_fast, macd_slow, macd_signal)
    d['MACD'] = macd.macd()
    d['MACD_Signal'] = macd.macd_signal()
    # Bollinger
    bb = ta.volatility.BollingerBands(d['close'], window=bb_window)
    d['BB_High'] = bb.bollinger_hband()
    d['BB_Low'] = bb.bollinger_lband()
    return d

def add_indicators_cpp(df, sma_window=20, ema_window=20, rsi_window=14, 
                       macd_fast=12, macd_slow=26, macd_signal=9, bb_window=20):
    """Our C++-based trading core version"""
    d = df.copy()
    close_prices = np.ascontiguousarray(d['close'].values, dtype=np.float64)
    
    d[f'SMA_{sma_window}'] = trading_core.calculate_sma(close_prices, sma_window)
    d[f'EMA_{ema_window}'] = trading_core.calculate_ema(close_prices, ema_window)
    d['RSI'] = trading_core.calculate_rsi(close_prices, rsi_window)
    
    macd_l, macd_s = trading_core.calculate_macd(close_prices, macd_fast, macd_slow, macd_signal)
    d['MACD'], d['MACD_Signal'] = macd_l, macd_s
    
    bb_h, bb_m, bb_l = trading_core.calculate_bollinger_bands(close_prices, bb_window, 2.0)
    d['BB_High'], d['BB_Low'] = bb_h, bb_l # bb_m es la SMA que ya tenemos
    
    return d

# --- Run Benchmark ---

print(f"\nStarting benchmark with {len(df_bench):,} rows...")

start = time.time()
res_py = add_indicators_python(df_bench)
t_py = time.time() - start
print(f"Python version (ta + pandas): {t_py:.4f}s")

start = time.time()
res_cpp = add_indicators_cpp(df_bench)
t_cpp = time.time() - start
print(f"C++ Core version:             {t_cpp:.4f}s")

print("-" * 30)
print(f"GLOBAL SPEEDUP: {t_py/t_cpp:.2f}x")

# --- RSI integrity check ---
np.testing.assert_allclose(res_py['RSI'].values[200:], res_cpp['RSI'].values[200:], rtol=1e-5)
print("\n✅ RSI check: OK")